In [9]:
import os
import shutil
import time
import pandas as pd
from tqdm import tqdm
import re
from pathlib import Path
import file_function as ff

In [10]:
                ###################
sell_company = "더키움자산관리대부"
                ###################
df_c = pd.read_excel(r'C:\Users\SL\Desktop\채무자조회.xlsx')
df_c = df_c[df_c.매각사==sell_company]   # df_c.loc[조건,:] 이렇게 안 해도 성능차이 없이 되네!!
df_c.head()

,채무자키,매각사구분,성명,주민번호,성별,매각사,회생사건결과,관리자기타,보증인성명,변경전성명,변경전주민번호,보증인수,파산사건결과
390,20426940,더키움,강경원,501230-1774714,남자,더키움자산관리대부,NaN,9008000005,NaN,NaN,NaN,0,NaN
393,20427731,더키움,강경전,720620-1552818,남자,더키움자산관리대부,NaN,9008000891,NaN,NaN,NaN,0,NaN
442,20428093,더키움,강근화,830511-2573611,여자,더키움자산관리대부,NaN,9008001302,NaN,NaN,NaN,0,NaN
464,20427314,더키움,강기욱,780529-1525015,남자,더키움자산관리대부,NaN,9008000427,NaN,NaN,NaN,0,NaN
527,20427858,더키움,강대선,801028-1093911,남자,더키움자산관리대부,NaN,9008001034,NaN,NaN,NaN,0,NaN


In [69]:
#[시작]##########################################################
# 1) 관리자번호와 하이픈은 지운다.
path = r'D:\1.다운로드\부산 공유폴더\3. (9008) 더키움자산관리대부_헬프\10. 초본\매입전 주민등록초본\1901'
f_d = r'D:\부산 정리\힣'
docu_kind = "초본"

In [70]:
os.chdir(path)
file_list = os.listdir(path)
index = 0
_ = "_"
key, name, name1, name2, resident, new_name, etc  = "", "", "", "", "", "", ""
admin_duple = []
no_search = []
changed = False

p1 = re.compile(r'\d{10}[-]') #지워버리자
p2 = re.compile(r"""  #이름이 2개인 것(개명)
(?P<name1>[가-힣]{2,})
(?P<resident>\d{6})
\W*
(?P<name2>[가-힣]{2,})
""", re.VERBOSE)
p3 = re.compile(r""" #이름이 1개인 것
(?P<name1>[가-힣]{2,})
(?P<resident>\d{6})
""", re.VERBOSE)

for f in file_list : 
    name_number = f
    ext = os.path.splitext(f)[1]
    if p1.search(f) :
        name_number = p1.sub("", name_number)
        print(name_number)
        
    if p2.search(name_number) :
        name1 = p2.search(name_number).group("name1")
        name2 = p2.search(name_number).group("name2")
        resident = p2.search(name_number).group("resident")

        
        cond1 = df_c.주민번호.str.contains(resident)    #참조불리언인덱싱 조건
        cond2 = df_c.성명.str.contains(name1)
        cond3 = df_c.성명.str.contains(name2)

        df_ = df_c[cond1&(cond2|cond3)]     

        if len(df_.index)==1 :              #참조검색 결과 1개?
            key = str(df_.채무자키.iloc[0]) 
            name = str(df_.성명.iloc[0])                                
            changed = True
            
        elif len(df_.index)>1 :
            admin_duple.append(f)           
        else :      #보증인에서 검색해보자
            cond2 = df_c.보증인성명.str.contains(name1)
            cond3 = df_c.보증인성명.str.contains(name2)
            df_ = df_c[cond1&(cond2|cond3)]

            if len(df_.index)==1 :             
                key = str(df_.채무자키.iloc[0]) 
                name = str(df_.보증인성명.iloc[0])
                etc = "보증인"                                
                changed = True
            else :
                no_search.append(f)            

    elif p3.search(name_number)  :
        name1 = p3.search(name_number).group("name1")
        resident = p3.search(name_number).group("resident")

        cond1 = df_c.주민번호.str.contains(resident)
        cond2 = df_c.성명.str.contains(name1)

        df_ = df_c[cond1&cond2]    

        if len(df_.index)==1 :              #참조검색 결과 1개?
            key = str(df_.채무자키.iloc[0]) 
            name = str(df_.성명.iloc[0])                                
            changed = True
            
        elif len(df_.index)>1 :
            admin_duple.append(f)           
        else :      #보증인에서 검색해보자
            cond2 = df_c.보증인성명.str.contains(name1)
            df_ = df_c[cond1&cond2]

            if len(df_.index)==1 :              
                key = str(df_.채무자키.iloc[0]) 
                name = str(df_.보증인성명.iloc[0])
                etc = "보증인"                                
                changed = True
            else :
                no_search.append(f)            

    else :
        print("정규식에 포함되지 않았다")


    if changed == True :
        if name2 != "" : 
            new_name = key + _ + name + _ + docu_kind
        else :
            new_name = key + _ + name + _ + docu_kind
        
        if etc != "" :
            new_name = new_name + _ + etc + ext
        else :
            new_name = new_name + ext



        temp = Path(f_d + "/"+ new_name)
        i = 1
        
        while os.path.exists(f_d+"/"+new_name): # os.path는 무조건 경로를 매개변수로..작업디렉토리가 아니므로 풀경로, new_name이 매개변수에 포함되어야 함!
            new_name = temp.stem + _ +"("+str(i)+")"+temp.suffix
            i += 1
                        
        f_d_final = f_d + "/" + new_name
        shutil.move(f, f_d_final)
        index += 1

print(index, "개 이동완료")
os.chdir('c:/')

# 2) 이름과 주민번호 앞자리를 분리하여 저장한다.
# 3) 참조df불러오고, 이름/보증인이름과 주민번호 앞자리로 함께 검색한다. (포함)



손미영690827-1.pdf
손미영690827-2.pdf
손미영690827.pdf
김영은590716.pdf
이현옥610307.pdf
이혜원810905.pdf
이윤성791202-1.pdf
이윤성791202.pdf
이임수-720614.pdf
정규식에 포함되지 않았다
이임수720614-1.pdf
이임수720614-2.pdf
이임수720614-3.pdf
이임수720614-4.pdf
이임수720614.pdf
최지혜760410(최지민).pdf
김명혜581018.pdf
김정화620515.pdf
윤세원710129.pdf
김종태650108-1.pdf
김종태650108.pdf
정일우690624.pdf
이태경721207.pdf
조우진810223.pdf
김선희700406.pdf
이창신620825.pdf
양혜숙631213-1.pdf
양혜숙631213.pdf
고선애680823.pdf
이승빈850715.pdf
조서현721005-1.pdf
조서현721005.pdf
손경자610305.pdf
이미자670201.pdf
강수경600712.pdf
김정헌790901.pdf
김혜경610304.pdf
정소라840617.pdf
박순자741223.pdf
강원준740312.pdf
김지수860205-1.pdf
김지수860205.pdf
박춘매660915.pdf
서지원760523-1.pdf
서지원760523-2.pdf
서지원760523-3.pdf
서지원760523-4.pdf
서지원760523.pdf
임수근801023.pdf
김숙자560903.pdf
송기덕780911.pdf
김태희760220.pdf
양기성640710.pdf
이진숙590702.pdf
정현택800503.pdf
서미숙-660217.pdf
정규식에 포함되지 않았다
서미숙660217.pdf
곽상윤700509-1.pdf
곽상윤700509.pdf
최광식820903.pdf
정병옥591203.pdf
백재남650717-1.pdf
백재남650717.pdf
안정숙770529.pdf
노재용660308.pdf
이현숙711102.pdf
배진우800102-1.pdf
배진우80